# ADU Property Data Extraction

Extract all properties with ADU or Accessory Dwelling mentions from county JSON files

In [1]:
import json
import pandas as pd
import os
import glob
import re
from datetime import datetime

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
# 定义数据目录路径
data_dir = "../data/raw/"

# 获取所有县的JSON文件
county_files = glob.glob(os.path.join(data_dir, "*_county_*.json"))
print(f"Found {len(county_files)} county JSON files")
for file in county_files:
    print(f"  - {os.path.basename(file)}")

Found 8 county JSON files
  - 20250822_114136_contra_costa_county_ca_contra_costa_county.json
  - 20250822_113940_los_angeles_county_ca_los_angeles_county.json
  - 20250822_114119_marin_county_ca_marin_county.json
  - 20250822_114033_san_francisco_county_ca_san_francisco_county.json
  - 20250822_114114_santa_clara_county_ca_santa_clara_county.json
  - 20250822_114101_san_mateo_county_ca_san_mateo_county.json
  - 20250822_114052_alameda_county_ca_alameda_county.json
  - 20250822_114021_orange_county_ca_orange_county.json


In [3]:
def extract_adu_properties(filepath):
    """
    从JSON文件中提取包含ADU或Accessory Dwelling的房产
    
    Args:
        filepath: JSON文件路径
    
    Returns:
        list: 包含ADU相关描述的房产列表
    """
    adu_properties = []
    
    # 定义ADU相关的关键词模式（不区分大小写）
    adu_pattern = re.compile(r'\b(adu|accessory\s+dwelling|granny\s+unit|in-?law\s+unit|secondary\s+unit)\b', re.IGNORECASE)
    
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
            
        # 获取县名
        county_name = data.get('metadata', {}).get('location', 'Unknown')
        
        # 遍历所有房产列表
        for listing in data.get('listings', []):
            # 获取描述文本
            description_text = listing.get('description', {}).get('text', '')
            
            # 检查是否包含ADU相关关键词
            if description_text and adu_pattern.search(description_text):
                # 提取关键信息
                property_data = {
                    # 基本信息
                    'county': county_name,
                    'listing_id': listing.get('listing_id', ''),
                    'status': listing.get('status', ''),
                    
                    # 地址信息
                    'address': listing.get('location', {}).get('address', {}).get('line', ''),
                    'city': listing.get('location', {}).get('address', {}).get('city', ''),
                    'state': listing.get('location', {}).get('address', {}).get('state_code', ''),
                    'zip': listing.get('location', {}).get('address', {}).get('postal_code', ''),
                    'lat': listing.get('location', {}).get('address', {}).get('coordinate', {}).get('lat', None),
                    'lon': listing.get('location', {}).get('address', {}).get('coordinate', {}).get('lon', None),
                    
                    # 价格信息
                    'price': listing.get('list_price', 0),
                    'price_per_sqft': listing.get('price_per_sqft', 0),
                    
                    # 房产详情
                    'beds': listing.get('description', {}).get('beds', 0),
                    'baths': listing.get('description', {}).get('baths', 0),
                    'sqft': listing.get('description', {}).get('sqft', 0),
                    'lot_sqft': listing.get('description', {}).get('lot_sqft', 0),
                    'year_built': listing.get('description', {}).get('year_built', 0),
                    'garage': listing.get('description', {}).get('garage', 0),
                    'property_type': listing.get('description', {}).get('type', ''),
                    
                    # ADU相关描述（保存完整描述以供后续分析）
                    'description': description_text,
                    
                    # 链接
                    'url': listing.get('href', ''),
                    
                    # 其他有用信息
                    'list_date': listing.get('list_date', ''),
                    'last_update': listing.get('last_update_date', ''),
                    'days_on_market': listing.get('description', {}).get('days_on_market', 0)
                }
                
                # 找出描述中匹配的ADU关键词
                matches = adu_pattern.findall(description_text)
                property_data['adu_keywords'] = ', '.join(set(matches))
                
                adu_properties.append(property_data)
                
    except Exception as e:
        print(f"Error processing {os.path.basename(filepath)}: {str(e)}")
    
    return adu_properties

In [4]:
# 处理所有县的文件并收集ADU相关房产
all_adu_properties = []
county_stats = []

for filepath in county_files:
    filename = os.path.basename(filepath)
    print(f"\nProcessing {filename}...")
    
    # 提取ADU房产
    adu_properties = extract_adu_properties(filepath)
    
    # 统计信息
    stats = {
        'file': filename,
        'adu_properties_count': len(adu_properties)
    }
    
    # 读取总房产数量（用于计算百分比）
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
            total_listings = len(data.get('listings', []))
            stats['total_listings'] = total_listings
            stats['adu_percentage'] = (len(adu_properties) / total_listings * 100) if total_listings > 0 else 0
    except:
        stats['total_listings'] = 0
        stats['adu_percentage'] = 0
    
    county_stats.append(stats)
    all_adu_properties.extend(adu_properties)
    
    print(f"  Found {len(adu_properties)} properties with ADU mentions")
    print(f"  Total listings: {stats['total_listings']}")
    print(f"  ADU percentage: {stats['adu_percentage']:.2f}%")

print(f"\n{'='*50}")
print(f"Total ADU properties found: {len(all_adu_properties)}")


Processing 20250822_114136_contra_costa_county_ca_contra_costa_county.json...
  Found 169 properties with ADU mentions
  Total listings: 1504
  ADU percentage: 11.24%

Processing 20250822_113940_los_angeles_county_ca_los_angeles_county.json...
  Found 861 properties with ADU mentions
  Total listings: 4513
  ADU percentage: 19.08%

Processing 20250822_114119_marin_county_ca_marin_county.json...
  Found 42 properties with ADU mentions
  Total listings: 330
  ADU percentage: 12.73%

Processing 20250822_114033_san_francisco_county_ca_san_francisco_county.json...
  Found 35 properties with ADU mentions
  Total listings: 370
  ADU percentage: 9.46%

Processing 20250822_114114_santa_clara_county_ca_santa_clara_county.json...
  Found 192 properties with ADU mentions
  Total listings: 1366
  ADU percentage: 14.06%

Processing 20250822_114101_san_mateo_county_ca_san_mateo_county.json...
  Found 78 properties with ADU mentions
  Total listings: 600
  ADU percentage: 13.00%

Processing 20250822_

In [5]:
# 创建主DataFrame
adu_df = pd.DataFrame(all_adu_properties)

# 显示基本信息
print(f"ADU DataFrame Shape: {adu_df.shape}")
print(f"\nColumns: {list(adu_df.columns)}")
print(f"\nData types:")
print(adu_df.dtypes)

ADU DataFrame Shape: (1890, 24)

Columns: ['county', 'listing_id', 'status', 'address', 'city', 'state', 'zip', 'lat', 'lon', 'price', 'price_per_sqft', 'beds', 'baths', 'sqft', 'lot_sqft', 'year_built', 'garage', 'property_type', 'description', 'url', 'list_date', 'last_update', 'days_on_market', 'adu_keywords']

Data types:
county             object
listing_id         object
status             object
address            object
city               object
state              object
zip                object
lat               float64
lon               float64
price               int64
price_per_sqft      int64
beds                int64
baths               int64
sqft              float64
lot_sqft          float64
year_built        float64
garage              int64
property_type      object
description        object
url                object
list_date          object
last_update        object
days_on_market      int64
adu_keywords       object
dtype: object


In [6]:
# 数据清理和类型转换
# 转换数值类型
numeric_columns = ['price', 'price_per_sqft', 'beds', 'baths', 'sqft', 'lot_sqft', 
                   'year_built', 'garage', 'days_on_market', 'lat', 'lon']

for col in numeric_columns:
    if col in adu_df.columns:
        adu_df[col] = pd.to_numeric(adu_df[col], errors='coerce')

# 转换日期类型
date_columns = ['list_date', 'last_update']
for col in date_columns:
    if col in adu_df.columns:
        adu_df[col] = pd.to_datetime(adu_df[col], errors='coerce')

# 计算一些有用的衍生指标
# 地块利用率（房屋面积/地块面积）
adu_df['lot_utilization'] = adu_df['sqft'] / adu_df['lot_sqft']
adu_df['lot_utilization'] = adu_df['lot_utilization'].replace([float('inf'), -float('inf')], 0)

# 剩余可建面积（粗略估计）
adu_df['remaining_lot_sqft'] = adu_df['lot_sqft'] - adu_df['sqft']

# 每间卧室的价格
adu_df['price_per_bedroom'] = adu_df['price'] / adu_df['beds']
adu_df['price_per_bedroom'] = adu_df['price_per_bedroom'].replace([float('inf'), -float('inf')], 0)

print("Data cleaning and feature engineering completed")

Data cleaning and feature engineering completed


In [7]:
# 显示前几行数据作为示例
print("\n" + "="*50)
print("Sample ADU Properties (First 5 rows)")
print("="*50)

# 选择关键列显示
display_columns = ['county', 'city', 'address', 'price', 'price_per_sqft', 
                   'sqft', 'lot_sqft', 'beds', 'baths', 'adu_keywords']

if len(adu_df) > 0:
    print(adu_df[display_columns].head())
else:
    print("No ADU properties found")


Sample ADU Properties (First 5 rows)
                    county         city          address    price  \
0  Contra Costa County, CA       Pinole    948 Jones Ave   899998   
1  Contra Costa County, CA        Rodeo    1343 Sixth St   700000   
2  Contra Costa County, CA      Concord  1950 Emerald St   850000   
3  Contra Costa County, CA       Albany   1235 Marin Ave  1150000   
4  Contra Costa County, CA  El Sobrante   615 Stanley Ln   975000   

   price_per_sqft    sqft  lot_sqft  beds  baths adu_keywords  
0               0  1651.0   17424.0     3      2          ADU  
1               0   775.0   10500.0     2      1          ADU  
2               0  1484.0    9750.0     4      2          ADU  
3               0  1510.0    5000.0     4      2          ADU  
4               0  2533.0   26001.0     5      3          ADU  


## Next Steps

现在你有了 `adu_df` DataFrame，包含了所有提及ADU的房产数据。你可以：

1. 进一步分析这些房产的特征
2. 筛选出最适合ADU开发的房产
3. 与zoning数据交叉验证
4. 计算潜在的投资回报率

In [10]:
#display adu_df.shape
adu_df.shape
#save to data/processed/adu_df.csv
adu_df.to_csv('../data/processed/adu_df.csv', index=False)